In [1]:
!pip install altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import altair as alt
import pandas as pd
from vega_datasets import data

In [4]:
alt.data_transformers.enable('default')

airportsData = "airports.csv"
routesData = "routes.csv"

airports = pd.read_csv(airportsData, header = 0)
routes = pd.read_csv(routesData, header = 0)

In [6]:
busyAirports = pd.concat([routes["source airport"], routes["destination airport"]])
busiestAirports = busyAirports.value_counts().rename_axis('airport').reset_index(name = 'number of flights')

airportLocations = airports[["iata", "latitude", "longitude"]]
airportLocations = airportLocations[airportLocations.iata != r'\N']
airportLocations = airportLocations.loc[airportLocations['iata'].isin(busiestAirports['airport'])]

busiestAirports = busiestAirports.merge(
    airportLocations,
    left_on=['airport'],
    right_on=['iata']
)

busiestAirports = busiestAirports[busiestAirports.airport == busiestAirports.iata]
busiestAirports = busiestAirports.iloc[:100]

In [7]:
worldMap = alt.topo_feature(data.world_110m.url, 'countries')

box = pd.DataFrame({'x1': [0], 'x2': [1600], 'y1': [0], 'y2': [900]})

bg = alt.Chart(box).mark_rect(fill='#aadaff', stroke='#aadaff').encode(
    alt.X('x1', scale=alt.Scale(domain=(0, 1600)), axis = None),
    alt.Y('y1', scale=alt.Scale(domain=(0, 900)), axis = None),
    x2='x2',
    y2='y2'
)

base = alt.Chart(worldMap).mark_geoshape(
    fill='#c3ecb2',
    stroke='#f5f8fb'
).properties(
    width = 1600,
    height = 900 
)

points = alt.Chart(busiestAirports).mark_circle(
    size=25,
    stroke='black',
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('number of flights:Q', scale=alt.Scale(scheme='yelloworangered'), title = ["Combined", "Arrivals/Departures"]),
    tooltip=['airport', 'number of flights']
)

map = bg + base + points

map.project('equirectangular').properties(title = 'A Map to show the 100 MOST used Airports').configure_title(fontSize = 25).configure_legend(titleFontSize = 18, gradientThickness = 30, gradientLength = 400, labelFontSize = 16, orient = 'bottom-left', titlePadding = 10)




alt.LayerChart(...)